#### Weekly Dataset

Each week, I'm dedicating myself to exploring, modeling, and doing stuff with a dataset. Trying to master different modalities and such while reviewing different statistical tests I don't use as often as I should.

This week, we're going morbid with Homecide data. You can find the dataset home <a href="https://www.kaggle.com/datasets/joebeachcapital/homicides">here</a> on kaggle. Shoutout to <a href="https://www.linkedin.com/in/joakim-arvidsson-7a2ab8/">Joakim Arvidsson</a> for providing the data on kaggle.

In [1]:
# standard DS helper guys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# for fancy plots
import plotly_express as px
import plotly.graph_objects as go

# atats stuff
import statsmodels.api as sm

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# read in file with correct encoding
df = pd.read_csv("/kaggle/input/homicides/homicide-data.csv",
                 encoding="latin1")

: 

## EDA and Clean Data

This is the way of a data scientist: look at data from summary stats. This can help see what you have and what questions you want to ask.

In [ ]:
df.head()

: 

In [ ]:
df.info()

: 

In [ ]:
df.describe()

: 

In [ ]:
df.isnull().sum()

: 

Null values are coming out of lat and lon and only one last name of a victim. Not bad. We can cross that if/when we need lat and lon.

In [ ]:
# uid is direct
len(df["uid"].unique()) == df.shape[0]

: 

In [ ]:
df["victim_race"].unique()

: 

In [ ]:
df["victim_age"].unique()

: 

In [ ]:
df["victim_sex"].unique()

: 

In [ ]:
# looks like only big cities are recorded
df["city"].unique()

: 

In [ ]:
# what's wI?
df["state"].unique()

: 

In [ ]:
# Wisconsin it is
df.loc[df["state"] == "wI"].head()

: 

Reported data seems to be in format YYYYMMDD, meaning YYYY-MM-DD. We need to slice that up to make it a date.

In [ ]:
# carve up date to be in format that is pandas readable
df["reported_date"] = df["reported_date"].apply(
    lambda x: f"{str(x)[:4]}-{str(x)[4:6]}-{str(x)[6:] if len(str(x)[6:]) == 2 else str(x)[7:]}"
)

df["reported_date"] = pd.to_datetime(df["reported_date"])

: 

In [ ]:
# spans all time from 2007 to 2017. Ten years worth of data
df["reported_date"].min(), df["reported_date"].max()

: 

In [ ]:
df["day_of_week"] = df["reported_date"].dt.day_of_week
df["month"] = df["reported_date"].dt.month

: 

In [ ]:
df.day_of_week.unique()

: 

In [ ]:
day_name_mapping = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

day_reverse_mapping = {v: k for k, v in day_name_mapping.items()}

: 

In [ ]:
df["day_of_week_name"] = df["day_of_week"].map(day_name_mapping)

: 

In [ ]:
day_homicides = df.groupby("day_of_week_name", as_index=False).count()[
    ["day_of_week_name", "uid"]
]

: 

In [ ]:
day_homicides["order"] = day_homicides["day_of_week_name"].map(
    day_reverse_mapping)

: 

In [ ]:
day_homicides.sort_values(by="order")

: 

In [ ]:
day_homicides.sort_values(by="order").rename(columns={"uid": "Homicide Count"}).plot(
    kind="bar",
    x="day_of_week_name",
    y="Homicide Count",
    xlabel="Day of Week",
    ylabel="Homicide Count",
    rot=45,
    title="Homicide Count by Day",
)

: 

In [ ]:
df["month_name"] = df["reported_date"].dt.month_name()

: 

In [ ]:
df.groupby(["month_name", "month"], as_index=False).count()[
    ["month_name", "month", "uid"]
].sort_values(by="month").rename(columns={"uid": "Homicide Count"}).plot(
    kind="bar",
    x="month_name",
    y="Homicide Count",
    xlabel="Month",
    ylabel="Homicide Count",
    title="Homicides by Month",
    rot=45,
)

: 

In [ ]:
df["victim_age"].min(), df["victim_age"].max()

: 

In [ ]:
df.loc[df["victim_age"] == "Unknown"].shape

: 

In [ ]:
# distribution of age when age is known
df.loc[df["victim_age"] != "Unknown"]["victim_age"].astype(int).hist()
plt.title("Victim Age Distribution")

: 

In [ ]:
df.query("victim_age != 'Unknown'").astype({"victim_age": int})[
    ["state", "victim_age"]
].groupby("state", as_index=False).mean().sort_values(by="victim_age", ascending=False)

: 

In [ ]:
df.groupby("victim_sex").count()[["uid"]]

: 

In [ ]:
# mainly men
df.groupby("victim_sex").count()[["uid"]] / df.shape[0]

: 

In [ ]:
# no unknowns for dispoition. This is a clear label
df["disposition"].unique()

: 

In [ ]:
df.groupby("disposition").count()["uid"]

: 

In [ ]:
df.groupby("disposition").count()["uid"] / df.shape[0]

: 

### Questions of Interest

1.) Which state has the highest number of homicides?<br>
2.) Is the homicide rate increasing or decreasing?<br>
3.) Does race or age or gender affect solve rate for a homicide?<br>
4.) Can we predict if a case will be solved?

## 1.) What State Has the Highest Number of Homicides?

For this, we'll do a simple group by then looks at counts in a nice map. This can help up the geo-data skills a bit.

In [ ]:
homicides_by_state = (
    df.groupby("state", as_index=False)
    .count()[["state", "victim_first"]]
    .sort_values(by="victim_first", ascending=False)
)

: 

In [ ]:
homicides_by_state

: 

In [ ]:
# upper case state to make merge easy
homicides_by_state["state"] = homicides_by_state["state"].str.upper()

: 

In [ ]:
# there are only 28 states, so we're missing a few here
df["state"].unique().shape

: 

In [ ]:
fig = go.Figure(
    data=go.Choropleth(
        locations=homicides_by_state["state"],
        z=homicides_by_state["victim_first"].astype(float),
        locationmode="USA-states",
        colorscale="Reds",
        colorbar_title="Homicides",
    )
)

fig.update_layout(
    title_text="Homicides from 2007-2017 by State",
    geo_scope="usa",  # limite map scope to USA
)

fig.show()

: 

This gives us a good idea of the total number, however, a better measure is usually the per capita rate. That's where we normalize or adjust based on the population. To do this, wee'll bring in a csv from github that has the state population data for 2014. That should general enough for us to use here.

In [ ]:
# read directly from github
pop_data = pd.read_csv(
    "https://gist.githubusercontent.com/bradoyler/0fd473541083cfa9ea6b5da57b08461c/raw/fa5f59ff1ce7ad9ff792e223b9ac05c564b7c0fe/us-state-populations.csv"
)

: 

In [ ]:
pop_data.head()

: 

In [ ]:
homicide_with_pop = homicides_by_state.merge(
    pop_data[["code", "pop_2014"]], left_on="state", right_on="code", how="inner"
)[["state", "victim_first", "pop_2014"]]

: 

In [ ]:
homicide_with_pop.head()

: 

The usual metrics is per 1000 people. Let's use that here, too.

In [ ]:
homicide_with_pop["homicide_per_100_000"] = (
    100_000 * homicide_with_pop["victim_first"] /
    (homicide_with_pop["pop_2014"])
)

: 

In [ ]:
homicide_with_pop.sort_values(by="homicide_per_100_000", ascending=False)

: 

In [ ]:
fig = go.Figure(
    data=go.Choropleth(
        locations=homicide_with_pop["state"],
        z=homicide_with_pop["homicide_per_100_000"].astype(float),
        locationmode="USA-states",
        colorscale="Reds",
        colorbar_title="Homicides per 100,000 Residents",
    )
)

fig.update_layout(
    title_text="Homicide Rate from 2007-2017 by State",
    geo_scope="usa",  # limite map scope to USA
)

fig.show()

: 

Considering these rate, DC is not where you want to be. However, this view give California a better shake: more people means more homicides, but a rate normalizes to reflect the reality of how often a murder is happening.

## 2.) Is the homicide rate increasing or decreasing?

For this, we have a simple time series chart. We only have 10 years worth of data, but worth a look. We'll do a simple less squares regression, see if it's statistically significant, and then plot it along with 

In [ ]:
# add a year column
df["year"] = df["reported_date"].dt.year

: 

In [ ]:
yearly_homicides = (
    df.groupby("year", as_index=False)
    .count()[["year", "victim_first"]]
    .rename(columns={"victim_first": "homicides"})
)

: 

In [ ]:
yearly_homicides.plot(
    kind="bar",
    x="year",
    y="homicides",
    title="Homicides by Year",
    xlabel="Year",
    ylabel="Total Homicides",
    rot=45,
)

: 

In [ ]:
X = sm.add_constant(yearly_homicides["year"])
y = yearly_homicides["homicides"]

: 

In [ ]:
model = sm.OLS(y, X)
results = model.fit()

: 

In [ ]:
print(results.summary())

: 

In [ ]:
results.params

: 

In [ ]:
yearly_homicides["fit_line"] = (
    yearly_homicides["year"] * results.params.year + results.params.const
)

: 

In [ ]:
yearly_homicides

: 

In [ ]:
fig, ax = plt.subplots()

ax.bar(yearly_homicides["year"], yearly_homicides["homicides"])
ax.plot(yearly_homicides["year"], yearly_homicides["fit_line"], c="red")

: 

I hesitate to say that this definiately shows that homicides are going up. AS you can see from the chart, 2016 is noticeably larger than the other points. It's possible that this slight outlier is causing the trend. Let's actually do something that's not extremely rigourous an take the make number, swap it with the middle number, re-run our regression. This might show that that year of homicides is just skewing us towards a positive curve.

In [ ]:
mid_year = yearly_homicides.shape[0] // 2
index_2016 = yearly_homicides.loc[
    yearly_homicides["year"] == 2016, "homicides"
].index.values[0]

: 

In [ ]:
mid_year, index_2016

: 

In [ ]:
yearly_homicides.loc[index_2016, "homicides"]

: 

In [ ]:
# swap 2016 val with mid value, this case, 2012
(
    yearly_homicides.loc[mid_year, "homicides"],
    yearly_homicides.loc[index_2016, "homicides"],
) = (
    yearly_homicides.loc[index_2016, "homicides"],
    yearly_homicides.loc[mid_year, "homicides"],
)

: 

In [ ]:
yearly_homicides

: 

In [ ]:
fig, ax = plt.subplots()

ax.bar(yearly_homicides["year"], yearly_homicides["homicides"])
ax.plot(yearly_homicides["year"], yearly_homicides["fit_line"], c="red")

: 

In [ ]:
X = sm.add_constant(yearly_homicides["year"])
y = yearly_homicides["homicides"]

model = sm.OLS(y, X)
results = model.fit()

: 

In [ ]:
yearly_homicides["fit_line"] = (
    yearly_homicides["year"] * results.params.year + results.params.const
)

: 

In [ ]:
fig, ax = plt.subplots()

ax.bar(yearly_homicides["year"], yearly_homicides["homicides"])
ax.plot(yearly_homicides["year"], yearly_homicides["fit_line"], c="red")

: 

In [ ]:
print(results.summary())

: 

So while our line is techically still going up, we see that our t-tests fail for this regression. So swapping the highest value with a mid-value destroies our significance.

## 3.) Does race or age or gender affect solve rate for a homicide?

This datasets contains whether or not homicide was solved or not. The proper labels are either open, closed by arrest, or closed without arrest.

We can assume closed without arrested and open are roughly the same. Open might be more recent cases. Let's see if open cases occur in the more recent years.

In [ ]:
df["disposition"].unique()

: 

In [ ]:
df.groupby(["disposition", "year"]).count()[["uid"]]

: 

So looks like closed without arrest is the most rare one. That makes sense. You don't want to close a case unless there's some particular reason. Maybe these cases show no one is at fault? Like self-defence cases or accidents? 

In order to not muddy the waters, we'll exlude that label from our tests and only look at open vs closed by arrest case. We want to use a statistical test to see if during this ten year period, the solve or close rate for cases between races, genders, or age is statisticall significant.

To do this, we'll use a chi-squared test since both our indepent variables (race, age, gender) are catgorical and our dependent variable (closed or not) is also categorical.

We'll start with race.

In [ ]:
df.loc[df["disposition"] != "Closed without arrest"].shape, df.shape

: 

In [ ]:
df_binary = df.loc[df["disposition"] != "Closed without arrest"]

: 

In [ ]:
pd.pivot_table(
    df_binary, index="victim_race", columns="disposition", values="uid", aggfunc="count"
)

: 

We want race to not be unknown or other. We want there to be a definitive label for the race of a victim, so we'll drop "other" and "unknow" from our data.

In [ ]:
df_binary = df_binary.loc[~df_binary["victim_race"].isin(["Other", "Unknown"])]

: 

In [ ]:
df_binary["victim_race"].unique()

: 

In [ ]:
pivot_table = pd.pivot_table(
    df_binary, index="victim_race", columns="disposition", values="uid", aggfunc="count"
)

: 

In [ ]:
pivot_table

: 

In [ ]:
# sum the rows and then divide each cell by row sum
pivot_table.apply(lambda x: x / x.sum(), axis=1)

: 

Just from a quick look, it appears black and hispanic homicides have a 50/50 solve rate, while white and Asian cases have a much higher rate. Close to 70%.

To make this statisically roboust though, scipy has a handy function called chi2_contingency. You feed it a contengency table then it spits out all your chi-squared stuff. 

Before we run the test though, we need to set our p-value. We'll do the conventional 0.05 for a p-value.

In [ ]:
from scipy.stats import chi2_contingency

: 

In [ ]:
chi2_contingency(pivot_table.values)

: 

The results her show us that closed rate does change by race. We can't say for certain what race is better or worse, but that race does factor into solve rate. 

To dig in a bit further, we can focus on two races and compare their average solve rates. Since black and white are our largest groups and those two groups make up a look of social debate, we'll run a t-test on those two groups. We'll randomly sample 100 instances from each group, calculate the average solve rate of each 100 sample pull. We'll do this 1000 times and compare.

In [ ]:
# create numeric variable for disposition
df_binary["disposition"] = df_binary["disposition"].map(
    lambda x: 1 if x == "Closed by arrest" else 0
)

: 

We'll set up our experiment as follows: we're randomly sample 100 homicides for both white and black victims. We take the average of the disposition aka the percent solved rate for those 100 samples. We do this 1000 times and do a t-test between the black and white group sample averages.

In [ ]:
white_solved = list()
black_solved = list()

for _ in range(1000):
    white_solved.append(
        df_binary.loc[df_binary["victim_race"] == "White"]
        .sample(100)["disposition"]
        .mean()
    )
    black_solved.append(
        df_binary.loc[df_binary["victim_race"] == "Black"]
        .sample(100)["disposition"]
        .mean()
    )

: 

In [ ]:
sample_data = pd.DataFrame(
    {
        "race": ["black" for _ in range(1000)] + ["white" for _ in range(1000)],
        "solved_average": black_solved + white_solved,
    }
)

: 

In [ ]:
sns.displot(data=sample_data, x="solved_average", hue="race", kind="kde")

: 

Just from our chart, we can see the average solve rate is much higher for white victims than black victims. To make this statistically clear though, we'll run a two way t-test.

In [ ]:
from statsmodels.stats.weightstats import ttest_ind

: 

In [ ]:
ttest_ind(white_solved, black_solved)

: 

The p-value is effectly zero. We can confidently reject the null hypothesis and say the solve rate is not the same for black and white victims.

Let's do the same for sex and see if we come to a similar conclusion.

In [ ]:
df["victim_sex"].unique()

: 

In [ ]:
df_binary_sex = df.loc[
    (df["victim_sex"] != "Unknown") & (
        df["disposition"] != "Closed without arrest")
]

: 

In [ ]:
pivot_table_sex = pd.pivot_table(
    df_binary_sex,
    index="victim_sex",
    columns="disposition",
    values="uid",
    aggfunc="count",
)

pivot_table_sex

: 

In [ ]:
# sum the rows and then divide each cell by row sum
pivot_table_sex.apply(lambda x: x / x.sum(), axis=1)

: 

Male victims are much less likely to be solved from our pivot table. However again, we'll put this to the test with a chi-squared test and a t-test for solved rates between men and woman.

We'll follow the same formulas from our previous experiments: p-values much be <= 0.05. For our t-test, we'll do 100 random samples average them and then repeat 1000 times.

In [ ]:
chi2_contingency(pivot_table_sex.values)

: 

In [ ]:
# create numeric variable for disposition
df_binary_sex["disposition"] = df_binary_sex["disposition"].map(
    lambda x: 1 if x == "Closed by arrest" else 0
)

: 

In [ ]:
men_solved = list()
women_solved = list()

for _ in range(1000):
    men_solved.append(
        df_binary_sex.loc[df_binary_sex["victim_sex"] == "Male"]
        .sample(100)["disposition"]
        .mean()
    )
    women_solved.append(
        df_binary_sex.loc[df_binary_sex["victim_sex"] == "Female"]
        .sample(100)["disposition"]
        .mean()
    )

: 

In [ ]:
sample_data_sex = pd.DataFrame(
    {
        "race": ["male" for _ in range(1000)] + ["female" for _ in range(1000)],
        "solved_average": men_solved + women_solved,
    }
)

: 

In [ ]:
sns.displot(data=sample_data_sex, x="solved_average", hue="race", kind="kde")

: 

In [ ]:
ttest_ind(men_solved, women_solved)

: 

Again, we get a p-value effectively of 0 for each test, meaning we can reject our null hypothesis that the solve rate for the two groups are the same.

Since we have some evidence for the solve rate being different for each group, we can move towards the best part: ML. We can make a classifier to see if we can predict whether a case will be solved or not.

## 4.) Can we predict if a case will be solved?

Having a decent size dataset (about 50,000 here) we can try an ML approach to homicide rate closure. We'll throw a few features together and then see what we can do.

We do have a lot of unknow values in terms of age, sex, and race. We'll leave those in there for now and see what happens. We're iterate after that.

In [ ]:
df_predict = df.loc[df["disposition"] != "Closed without arrest"]

: 

In [ ]:
# we start parsing out variable we think are interesting. We'll exclude year
# since we want to predict based on these ten years worth of data regardless of that
df_predict = df_predict[
    [
        "victim_race",
        "victim_age",
        "victim_sex",
        "state",
        "day_of_week",
        "month",
        "disposition",
    ]
]

: 

In [ ]:
# luckily our labels are balanced. Our baseline is an accuracy better than 52% for predicting an arrest
df_predict["disposition"].value_counts() / df_predict.shape[0]

: 

In [ ]:
numeric_variables = ["victim_age"]
categorical_variables = ["victim_race",
                         "victim_sex", "state", "day_of_week", "month"]

: 

In [ ]:
# change unknown age to be large negative number
df_predict["victim_age"] = (
    df_predict["victim_age"].map(
        lambda x: -999 if x == "Unknown" else x).astype(int)
)

: 

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, auc, confusion_matrix

: 

In [ ]:
df_predict.shape

: 

In [ ]:
# before anything, create a split
X_train, X_validation, y_train, y_validation = train_test_split(
    df_predict.drop("disposition", axis=1),
    df_predict["disposition"],
    test_size=0.3,
    random_state=42,
    stratify=df_predict["disposition"],
)

: 

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(
    X_validation, y_validation, test_size=0.5, random_state=42, stratify=y_validation
)

: 

In [ ]:
col_transformer = ColumnTransformer(
    [
        (
            "OneHot",
            OneHotEncoder(drop="first", handle_unknown="ignore"),
            categorical_variables,
        ),
        ("MinMax", MinMaxScaler(), numeric_variables),
    ]
)

: 

In [ ]:
pipeline = Pipeline(
    [("tranfromers", col_transformer), ("model", LogisticRegressionCV())]
)

: 

In [ ]:
pipeline.fit(X_train, y_train)

: 

In [ ]:
accuracy_score(pipeline.predict(X_val), y_val)

: 

In [ ]:
confusion_matrix(pipeline.predict(X_val), y_val)

: 

Not great, but better than guessing. The age variable might have affected our linear logistic regression model. Let's try some non-linear models here like random forest or catboost.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

: 

In [ ]:
pipeline_rf = Pipeline(
    [("tranfromers", col_transformer), ("model", RandomForestClassifier())]
)

: 

In [ ]:
pipeline_rf.fit(X_train, y_train)

: 

In [ ]:
accuracy_score(pipeline_rf.predict(X_val), y_val)

: 

In [ ]:
confusion_matrix(pipeline_rf.predict(X_val), y_val)

: 

In [ ]:
col_transformer_cb = ColumnTransformer(
    [("MinMax", MinMaxScaler(), numeric_variables)], remainder="passthrough"
)

pipeline_cb = Pipeline(
    [
        ("tranforms", col_transformer_cb),
        ("model", CatBoostClassifier(
            verbose=False, cat_features=[1, 2, 3, 4, 5])),
    ]
)

: 

In [ ]:
pipeline_cb.fit(X_train, y_train)

: 

In [ ]:
accuracy_score(pipeline_cb.predict(X_val), y_val)

: 

In [ ]:
confusion_matrix(pipeline_cb.predict(X_val), y_val)

: 

CatBoost seems the best. We can use CatBoost and Random Forest Models to get an idea of what our models are focusing on via the feature_importances_ method.

In [ ]:
pipeline_cb["model"].feature_importances_

: 

In [ ]:
pipeline_rf["model"].feature_importances_

: 

In [ ]:
variable_importance = X_train.copy()
variable_importance["day_of_week"] = variable_importance["day_of_week"].astype(
    str)
variable_importance["month"] = variable_importance["month"].astype(str)

: 

In [ ]:
dummy_variables = pd.get_dummies(
    variable_importance[categorical_variables], drop_first=True
).columns.tolist()

: 

In [ ]:
dummy_variables += ["age"]

: 

In [ ]:
fix, ax = plt.subplots(figsize=(15, 10))
ax.bar(dummy_variables, pipeline_rf["model"].feature_importances_)
_ = plt.xticks(rotation=90), plt.title("Feature Importance")

: 

In [ ]:
# cat boost can take categorical columns as features. Here, we rearrange the columns to be in the correct order
# that we passed them to catboost
cols = X_train.columns.tolist()
age_col = cols.pop(1)
df_cols = [age_col] + cols

fix, ax = plt.subplots(figsize=(15, 10))
ax.bar(df_cols, pipeline_cb["model"].feature_importances_)
_ = plt.xticks(rotation=90), plt.title("Feature Importance")

: 

The Random Forest tends to really focus on age while giving other features less importance. Catboost on the other hand, uses state more often. Note also that catboost can give feature importance over entire categorical columns, not just the dummy column variable features.

Over these three models though, we're barely getting better than random guessing. We are including the unknown data. Let's see how many of those we have across all variables.

In [ ]:
X_train_no_unknown = X_train.loc[
    (X_train["victim_race"] != "Unknown")
    | (X_train["victim_sex"] != "Unknown")
    | (X_train["victim_age"] != -999)
]

: 

In [ ]:
X_train_no_unknown.shape

: 

In [ ]:
y_train_no_unknown = y_train.loc[y_train.index.isin(
    X_train_no_unknown.index.tolist())]

: 

In [ ]:
y_train_no_unknown.shape

: 

In [ ]:
pipeline_cb.fit(X_train_no_unknown, y_train_no_unknown)

: 

In [ ]:
X_val_no_unknown = X_val.loc[
    (X_val["victim_race"] != "Unknown")
    | (X_val["victim_sex"] != "Unknown")
    | (X_val["victim_age"] != -999)
]

y_val_no_unknown = y_val.loc[y_val.index.isin(X_val_no_unknown.index.tolist())]

: 

In [ ]:
accuracy_score(pipeline_cb.predict(X_val), y_val)

: 

In [ ]:
pipeline.fit(X_train_no_unknown, y_train_no_unknown)

: 

In [ ]:
accuracy_score(pipeline.predict(X_val_no_unknown), y_val_no_unknown)

: 

In [ ]:
pipeline_rf.fit(X_train_no_unknown, y_train_no_unknown)

: 

In [ ]:
accuracy_score(pipeline_rf.predict(X_val_no_unknown), y_val_no_unknown)

: 

Removing the unknowns didn't really do much for us. What we can try here is optimizing our hyperparameters. It might not get us a ton, but it could maybe get us to 70%? That might be a bit optimistic but we can try.

We must also remember that this might not be ML solvable problem with the data we have. There is a lot of variation between homicides. Some could be open and closed cases easily solved. Other could be more difficult that are gang related or the victim had no connection to the murderer. Heading into anything ML, you gotta keep in mind that we might be able to do ML to predict it.

Anywho, let's use industry optuna to do this. It's the fairly standard way to tune sklearn models. We'll do 100 trials with optuna, choosing some key hyperparameters from catboost.

In [ ]:
y_train = y_train.map(lambda x: 1 if x == "Closed by arrest" else 0)
y_val = y_val.map(lambda x: 1 if x == "Closed by arrest" else 0)

: 

In [ ]:
import optuna


def objective(trial):
    param = {
        "objective": trial.suggest_categorical(
            "objective", ["Logloss", "CrossEntropy"]
        ),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "boosting_type": trial.suggest_categorical(
            "boosting_type", ["Ordered", "Plain"]
        ),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float(
            "bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    col_transformer_cb = ColumnTransformer(
        [("MinMax", MinMaxScaler(), numeric_variables)], remainder="passthrough"
    )

    pipeline_cb = Pipeline(
        [
            ("tranforms", col_transformer_cb),
            (
                "model",
                CatBoostClassifier(
                    verbose=False, cat_features=[1, 2, 3, 4, 5], **param
                ),
            ),
        ]
    )

    pipeline_cb.fit(X_train, y_train)

    preds = pipeline_cb.predict(X_val)
    accuracy = accuracy_score(y_val, preds)
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

: 

From our 100 trials, we were barely able to get out of the 62% range. Might be that this problem can't really be solved via the ML solutions we have now. Might be worth future attempts to create DNN's (deep neural networks), but for now, I'm capping this and putting a bow on it.